# ML Inference Pipeline Example

This notebook demonstrates batch inference using a trained model.
It will be executed automatically by AWS Batch using Papermill.

In [ ]:
# Parameters (can be overridden by Papermill)
input_bucket = "my-ml-input"
model_key = "models/model.pth"
data_key = "data/inference_data.csv"
batch_size = 1000

In [ ]:
import os
import boto3
import pandas as pd
import numpy as np
import torch
from datetime import datetime

# Get environment variables from Batch job
OUTPUT_BUCKET = os.environ.get('OUTPUT_BUCKET', input_bucket)
OUTPUT_PREFIX = os.environ.get('OUTPUT_PREFIX', 'results/')

s3_client = boto3.client('s3')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Load Model from S3

In [ ]:
# Download model
os.makedirs('/workspace/models', exist_ok=True)
model_path = '/workspace/models/model.pth'

print(f"Downloading model from s3://{input_bucket}/{model_key}")
s3_client.download_file(input_bucket, model_key, model_path)

# Load model checkpoint
checkpoint = torch.load(model_path, map_location=device)
print(f"Model loaded successfully")
print(f"Model configuration: input_size={checkpoint['input_size']}, "
      f"hidden_size={checkpoint['hidden_size']}, num_classes={checkpoint['num_classes']}")

## Load Inference Data

In [ ]:
# Download data
data_path = '/workspace/inference_data.csv'
print(f"Downloading data from s3://{input_bucket}/{data_key}")
s3_client.download_file(input_bucket, data_key, data_path)

# Load and prepare data
df = pd.read_csv(data_path)
print(f"Loaded {len(df)} samples")
print(f"\nData shape: {df.shape}")
df.head()

## Run Batch Inference

In [ ]:
# Reconstruct model (you'll need to define your model class)
# For this example, we'll use a simple approach

class SimpleNeuralNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size // 2)
        self.fc3 = torch.nn.Linear(hidden_size // 2, num_classes)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

model = SimpleNeuralNetwork(
    checkpoint['input_size'],
    checkpoint['hidden_size'],
    checkpoint['num_classes']
).to(device)

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("Model ready for inference")

In [ ]:
# Prepare input features (assuming numeric columns)
# Adjust based on your actual data
feature_columns = [col for col in df.columns if col != 'id']
X = df[feature_columns].values

# Normalize using saved scaler
scaler = checkpoint['scaler']
X_scaled = scaler.transform(X)

# Convert to tensor
X_tensor = torch.FloatTensor(X_scaled).to(device)

print(f"Running inference on {len(X_tensor)} samples...")

# Batch inference
predictions = []
probabilities = []

with torch.no_grad():
    for i in range(0, len(X_tensor), batch_size):
        batch = X_tensor[i:i+batch_size]
        outputs = model(batch)
        probs = torch.softmax(outputs, dim=1)
        _, preds = torch.max(outputs, 1)
        
        predictions.extend(preds.cpu().numpy())
        probabilities.extend(probs.cpu().numpy())
        
        if (i // batch_size + 1) % 10 == 0:
            print(f"Processed {i + len(batch)}/{len(X_tensor)} samples")

print(f"Inference completed!")

## Save Results

In [ ]:
# Create results directory
os.makedirs('/workspace/output', exist_ok=True)

# Add predictions to dataframe
df['prediction'] = predictions

# Add probability columns
prob_array = np.array(probabilities)
for i in range(prob_array.shape[1]):
    df[f'prob_class_{i}'] = prob_array[:, i]

# Save results
results_path = '/workspace/output/predictions.csv'
df.to_csv(results_path, index=False)
print(f"Results saved to {results_path}")

# Display sample results
df.head(10)

## Generate Summary Statistics

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Prediction distribution
plt.figure(figsize=(10, 6))
sns.countplot(x='prediction', data=df)
plt.title('Prediction Distribution')
plt.xlabel('Predicted Class')
plt.ylabel('Count')
plt.savefig('/workspace/output/prediction_distribution.png')
plt.show()

# Confidence distribution
max_probs = prob_array.max(axis=1)
plt.figure(figsize=(10, 6))
plt.hist(max_probs, bins=50, edgecolor='black')
plt.title('Prediction Confidence Distribution')
plt.xlabel('Max Probability')
plt.ylabel('Count')
plt.savefig('/workspace/output/confidence_distribution.png')
plt.show()

print(f"\nSummary Statistics:")
print(f"Total samples: {len(df)}")
print(f"\nPrediction counts:")
print(df['prediction'].value_counts())
print(f"\nAverage confidence: {max_probs.mean():.4f}")
print(f"Min confidence: {max_probs.min():.4f}")
print(f"Max confidence: {max_probs.max():.4f}")

## Upload Results to S3

In [ ]:
# This will be automatically handled by the container entrypoint script
# But we can also explicitly upload specific files

output_key_prefix = OUTPUT_PREFIX

files_to_upload = [
    'predictions.csv',
    'prediction_distribution.png',
    'confidence_distribution.png'
]

for filename in files_to_upload:
    local_path = f'/workspace/output/{filename}'
    s3_key = f'{output_key_prefix}{filename}'
    
    if os.path.exists(local_path):
        s3_client.upload_file(local_path, OUTPUT_BUCKET, s3_key)
        print(f"Uploaded s3://{OUTPUT_BUCKET}/{s3_key}")

print(f"\n✅ Inference completed successfully!")
print(f"📊 Results available at: s3://{OUTPUT_BUCKET}/{output_key_prefix}")